In [ ]:
CMD = "wget https://raw.githubusercontent.com/Diyago/"\
  "GAN-for-tabular-data/master/requirements.txt"

!{CMD}
!pip install -r requirements.txt
!pip install tabgan

--2024-05-02 13:23:55--  https://raw.githubusercontent.com/Diyago/GAN-for-tabular-data/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 214 [text/plain]
Saving to: ‘requirements.txt.1’

requirements.txt.1  100%[===================>]     214  --.-KB/s    in 0s      

2024-05-02 13:23:55 (3.69 MB/s) - ‘requirements.txt.1’ saved [214/214]



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

In [ ]:
df = pd.read_csv("data.csv")
df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0,0.000000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0,0.002770
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0,0.005540
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0,0.008310
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0,0.011080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,0.477011,0.250000,0.0,0.0,0.686747,0.587312,0.782917,0.0,...,0.0,0.750000,0.0,0.0,0.271318,0.109500,4,1,2,0.540166
20627,100,197,0.408046,0.083333,0.0,0.0,0.701807,0.729453,0.866475,0.0,...,0.0,0.583333,0.0,0.0,0.124031,0.366197,3,1,2,0.542936
20628,100,198,0.522989,0.500000,0.0,0.0,0.665663,0.684979,0.775321,0.0,...,0.0,0.833333,0.0,0.0,0.232558,0.053991,2,1,2,0.545706
20629,100,199,0.436782,0.750000,0.0,0.0,0.608434,0.746021,0.747468,0.0,...,0.0,0.583333,0.0,0.0,0.116279,0.234466,1,1,2,0.548476


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20631 entries, 0 to 20630
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          20631 non-null  int64  
 1   cycle       20631 non-null  int64  
 2   setting1    20631 non-null  float64
 3   setting2    20631 non-null  float64
 4   setting3    20631 non-null  float64
 5   s1          20631 non-null  float64
 6   s2          20631 non-null  float64
 7   s3          20631 non-null  float64
 8   s4          20631 non-null  float64
 9   s5          20631 non-null  float64
 10  s6          20631 non-null  float64
 11  s7          20631 non-null  float64
 12  s8          20631 non-null  float64
 13  s9          20631 non-null  float64
 14  s10         20631 non-null  float64
 15  s11         20631 non-null  float64
 16  s12         20631 non-null  float64
 17  s13         20631 non-null  float64
 18  s14         20631 non-null  float64
 19  s15         20631 non-nul

In [ ]:
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    df.drop("RUL", axis=1),
    df["RUL"],
    test_size=0.20,
    #shuffle=False,
    random_state=42,
)

In [ ]:
# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), \
  df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

In [ ]:
# Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

In [ ]:
# Build the neural network
model = Sequential()
# Hidden 1
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(12, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor], verbose=2,epochs=1000)

Epoch 1/1000
516/516 - 3s - loss: 5360.0210 - val_loss: 2191.3271 - 3s/epoch - 5ms/step
Epoch 2/1000
516/516 - 1s - loss: 1755.0685 - val_loss: 1510.9005 - 1s/epoch - 2ms/step
Epoch 3/1000
516/516 - 1s - loss: 1506.1343 - val_loss: 1426.4415 - 1s/epoch - 2ms/step
Epoch 4/1000
516/516 - 2s - loss: 1441.1337 - val_loss: 1620.4131 - 2s/epoch - 3ms/step
Epoch 5/1000
516/516 - 2s - loss: 1421.4690 - val_loss: 1363.2379 - 2s/epoch - 3ms/step
Epoch 6/1000
516/516 - 1s - loss: 1407.9423 - val_loss: 1424.9016 - 1s/epoch - 3ms/step
Epoch 7/1000
516/516 - 1s - loss: 1370.5559 - val_loss: 1351.9376 - 1s/epoch - 2ms/step
Epoch 8/1000
516/516 - 1s - loss: 1379.2113 - val_loss: 1354.4696 - 1s/epoch - 2ms/step
Epoch 9/1000
516/516 - 1s - loss: 1372.2456 - val_loss: 1350.6901 - 1s/epoch - 2ms/step
Epoch 10/1000
516/516 - 1s - loss: 1354.1473 - val_loss: 1351.1415 - 1s/epoch - 2ms/step
Epoch 11/1000
516/516 - 1s - loss: 1368.8938 - val_loss: 1481.9741 - 1s/epoch - 2ms/step
Epoch 12/1000
516/516 - 1s - l

In [ ]:
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

129/129 [==============================] - 0s 1ms/step
Final score (RMSE): 35.99531149604721


In [ ]:
!pip install tabgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux

In [ ]:
from tabgan.sampler import GANGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, \
              is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100,
               "learning_rate": 0.02, "random_state": \
                42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,\
           gen_params = {"batch_size": 500, "patience": 25, \
          "epochs" : 500,}).generate_data_pipe(df_x_train, df_y_train,\
          df_x_test, deep_copy=True, only_adversarial=False, \
          use_adversarial=True)



Fitting CTGAN transformers for each column:   0%|          | 0/30 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 3302, number of negative: 3301
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001016 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1600
[LightGBM] [Info] Number of data points in the train set: 6603, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500076 -> initscore=0.000303
[LightGBM] [Info] Start training from score 0.000303
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [ ]:
gen_x

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,label1,label2,cycle_norm
0,1,187,0.229885,0.500000,0.0,0.0,0.635542,0.459124,0.759959,0.0,...,0.757599,0.0,0.666667,0.0,0.0,0.217054,0.259597,1,2,0.515235
1,6,58,0.201149,0.166667,0.0,0.0,0.487952,0.325485,0.542539,0.0,...,0.392459,0.0,0.416667,0.0,0.0,0.589147,0.401270,0,0,0.157895
2,1,176,0.385057,0.250000,0.0,0.0,0.496988,0.462830,0.683322,0.0,...,0.617160,0.0,0.500000,0.0,0.0,0.449612,0.490058,1,1,0.484765
3,1,168,0.252874,0.666667,0.0,0.0,0.442771,0.439285,0.564652,0.0,...,0.644863,0.0,0.416667,0.0,0.0,0.480620,0.528583,1,1,0.462604
4,8,98,0.224138,0.750000,0.0,0.0,0.490964,0.449095,0.543214,0.0,...,0.601770,0.0,0.416667,0.0,0.0,0.441860,0.524993,0,0,0.268698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15977,70,134,0.431034,0.166667,0.0,0.0,0.867470,0.759538,0.809588,0.0,...,0.799923,0.0,0.750000,0.0,0.0,0.108527,0.234051,1,2,0.368421
15978,83,12,0.591954,0.333333,0.0,0.0,0.069277,0.336385,0.334740,0.0,...,0.152751,0.0,0.250000,0.0,0.0,0.596899,0.766777,0,0,0.030471
15979,5,267,0.603448,0.583333,0.0,0.0,0.759036,0.836494,0.810263,0.0,...,0.786072,0.0,0.750000,0.0,0.0,0.271318,0.224247,1,2,0.736842
15980,6,183,0.557471,0.916667,0.0,0.0,0.843373,0.583824,0.813977,0.0,...,0.786841,0.0,0.666667,0.0,0.0,0.147287,0.183789,1,2,0.504155


In [ ]:
# Predict
pred = model.predict(gen_x.values)
score = np.sqrt(metrics.mean_squared_error(pred,gen_y.values))
print("Final score (RMSE): {}".format(score))

500/500 [==============================] - 1s 2ms/step
Final score (RMSE): 36.1427850040103
